In [1]:
from pathlib import Path
import os
import sys 
path_dcf = str(Path(os.path.abspath('.')).resolve().parent)
sys.path.append(path_dcf)
from config import *
from algorithms.oneshot_dcf import DCF
from utils import *
from algorithms.utils_dilated_tubules import *
import pandas as pd
import tifffile
import matplotlib.pyplot as plt
import cv2

/pasteur/appa/homes/ahabis/miniconda3/envs/scribble/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/pasteur/appa/homes/ahabis/miniconda3/envs/scribble/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/local/scratch/tmp/ipykernel_1160442/3382141517.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but wa

## Choose a Slide from the dataset and a dilated tubule from the given slide

In [2]:
nb_slide = 1
nb_support_dilated_tubule = 8
nb_query_dilated_tubule = 20

### Instanciate the object DCF one-shot

In [3]:
dcf = DCF(n_epochs=100,
            nb_augment=100,
            isolines=np.array([0.0, 1.0]),
            learning_rate=0.,
            clip=1e-1,
            sigma=5,
            weights=0.9,
            exponential_decay=0.999,
            thresh=1e-2,
        )

/pasteur/appa/homes/ahabis/miniconda3/envs/scribble/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


### Show the Support dilated tubule and the annotation of the support

In [4]:
pentagon_coords = torch.tensor([[
    [[1.0, 0.0],            # Vertex 1
     [0.309, 0.951],        # Vertex 2
     [-0.809, 0.588],       # Vertex 3
     [-0.809, -0.588],      # Vertex 4
     [0.309, -0.951]]       # Vertex 5
]])

scaled_pentagon_coords = (pentagon_coords * 0.5) + 0.5
scaled_pentagon_coords = scaled_pentagon_coords.view(1, 1, 5, 2)
contour_support = scaled_pentagon_coords.cuda()
numpy_array = contour_support.squeeze().cpu().detach().numpy()
# plt.plot(numpy_array[:,0], numpy_array[:,1])
# plt.show()

In [5]:
img_support = torch.randn(1*3*512*512).reshape((1,3,512,512)).cuda().to(torch.float32)
print(img_support.dtype)

torch.float32


### Fit the one shot DCF

In [6]:
dcf.fit(img_support,
        contour_support,
        augment = True)

100%|██████████| 100/100 [00:01<00:00, 76.17it/s]


### Predict on the query

In [7]:
# filename_img = row_to_filename(annotations.iloc[nb_query_dilated_tubule])
# img_test = tifffile.imread(os.path.join(path_images,filename_img))
# contour_init = contour_inits[filename_img]
# contour_init = preprocess_contour(contour_init,img_test)
contours, score, tots, energies = dcf.predict(img_support,
                                               contour_support)
# tots[tots == 0] = 1e100
# x = np.argmin(tots)
# contour_to_display = (np.expand_dims(contours[-1], 1)).astype(int)
# new_img = cv2.drawContours((((img_test/np.max(img_test))*255).astype(np.uint8)).copy(),
#                         [contour_to_display],
#                         0,
#                         color = 1,
#                         thickness=2)
# plt.imshow(new_img)
# plt.show()


/pasteur/appa/homes/ahabis/miniconda3/envs/scribble/lib/python3.9/site-packages/torch_contour/torch_contour.py:511: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  smoothed_tensor = F.conv1d(out_moved_axis, self.kernel, padding="same", groups=2)


### This cell save the images of the evolution of the contour over time in folder_images_paper

In [ ]:
import matplotlib
cmap = matplotlib.cm.get_cmap('cool')

colors = [cmap(u)[:-1] for u in np.linspace(0,1,dcf.n_epochs+1)]
colors = (np.array(colors)*255).astype(np.int32).tolist()

tots = []
imgs = []

for i, contour in enumerate(contours):

    contour_to_display = (np.expand_dims(contour, 1)).astype(int)
    
    new_img = cv2.drawContours((((img_test/np.max(img_test))*255).astype(np.uint8)).copy(),
                               [contour_to_display],
                               0,
                               color = colors[0],
                               thickness=3)
    plt.imsave(os.path.join('folder_images_paper',filename_img.split('.')[0]+'_'+str(i)+'.png'),new_img)


### This cell plot the evolution of the energies by (isoline, scale) over time

In [1]:
# ener = np.squeeze(energies.cpu().detach().numpy())                                                
# isolines = dcf.isolines.cpu().detach().numpy()
# for scale in range(5):
#     for index, isoline in enumerate(isolines):
#         plt.plot(np.arange(dcf.n_epochs),ener[:,scale,index],label = 'isoline '+str(isoline))
#     plt.title('evolution of the energies at scale: ' +str(scale+1))
#     plt.xlabel('step')
#     plt.ylabel('energie value')
#     plt.legend()
#     plt.show()


### This cell plot the repartition of the annotation over the different slides

In [ ]:
# annotations = pd.read_csv(os.path.join(path_annotations,'annotations.csv'), index_col=0)
# annotations = annotations.replace(['dilated_tubule','fake_tubule'], [1,0])
# dt_annotations = annotations[['image','term']].groupby('image').sum()
# dt_annotations.columns = ['number of dilated tubules']
# all_annotations = annotations[['image','term']].groupby('image').count()
# all_annotations.columns = ['number of annotations']

# dataset_stats = pd.concat([dt_annotations,all_annotations], axis = 1)
# dataset_stats['number of false dilated_tubule'] = dataset_stats['number of annotations'] - dataset_stats['number of dilated tubules']
# dataset_stats['image'] = dataset_stats.index
# dataset_stats = dataset_stats.reset_index(drop = True)
# dataset_stats['image'] = dataset_stats['image'].apply(lambda x: x.split('.')[0])
# dataset_stats
# # dataset_stats['image'] = dataset_stats.index.apply(lambda x : x.split('.')[0])

# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set(font_scale=1)

# plt.figure(figsize = (10,10))
# # plotting columns

# ax = sns.barplot(x=dataset_stats['image'],
#                  y=dataset_stats['number of annotations'],
#                  color = np.array([207,105,138])/255,
#                  label = 'n° of annotations')

# ax = sns.barplot(x=dataset_stats['image'],
#                  y=dataset_stats['number of dilated tubules'],
#                  color = np.array([137,44,80])/255,
#                  label = 'n° of dilated tubules')
 
# # renaming the axes
# ax.set_xlabel("Slide",fontsize = 20)
# ax.set_ylabel("N° of instances",fontsize = 20)
# plt.legend(fontsize = 20)

# # visualizing illustration
# plt.show()
